In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import datetime
import torch
from torch.nn import functional as F
import scipy.stats
import sklearn.datasets

import sys
sys.path.append("../")
import curvvae_lib.train.predictive_passthrough_trainer as ppttrainer
import curvvae_lib.architecture.passthrough_vae as ptvae
import curvvae_lib.architecture.save_model as sm
import curvvae_lib.architecture.load_model as lm

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-51yqtr63 because the default path (/home/tsr42/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# Dataset

In [2]:
foodname = "banana"
foldername = f"fork_trajectory_{foodname}"
savefilename = f"{foodname}_clean_pickups"

In [3]:
train = []
training_ts = np.linspace(0,1,64)
attempt = 1
while True:
    try:
        raw_vals = np.load(f"{savefilename}/pickup_attempt{attempt}.npy")
        train.append(raw_vals.T.flatten())
    except:
        print(f"We found {attempt-1} pickup attempts")
        break
    attempt += 1

train = np.array(train).reshape(-1,7,64)
all_points = train[:,:,:]

time_shape = list(all_points.shape)
time_shape[1] = 1
# why be smart when you can be dumb
t = np.ones(time_shape)
for i in range(time_shape[2]):
    t[:,:,i] = t[:,:,i] * i / (time_shape[2] + 0.0)

all_points = np.concatenate((t, all_points), axis=1)
all_points = all_points.transpose(0,2,1)
# actually this is all points, not just start points
start_points = all_points[:,:,1:].reshape(-1,7)
print("start_points_shape", start_points.shape)
# See http://localhost:8889/notebooks/scratchwork/2021-09-17%20Rotation%20Scaling.ipynb
# for why we want quaternion values to be multiplied by 0.16 when position values are in meters 
# (if the relevant distance scale of the fork is 0.08 meters, ie: 8cm).
mean = np.mean(start_points, axis=0)
variance = np.var(start_points, axis=0)
position_std = np.sqrt(np.max(variance))
print("std of: ", position_std)
position_scaling = 1/position_std
rotation_scaling = 0.16 * position_scaling

start_points = start_points - mean
poss = position_scaling
rts = rotation_scaling
start_points = (start_points * np.array((poss,poss,poss,rts,rts,rts,rts)))
#start_points = start_points[:,3:]
print("shape before reducing dimension", all_points.shape)
print("dataset variance of:", np.var(start_points,axis=0))

We found 155 pickup attempts
start_points_shape (9920, 7)
std of:  0.17859106200728153
shape before reducing dimension (155, 64, 8)
dataset variance of: [0.00082463 0.00106736 0.00316102 0.00634536 0.0256     0.00564557
 0.00894256]


In [4]:
dataset = start_points
dataset

array([[ 5.30630546e-02, -7.39897730e-02,  6.99848376e-02, ...,
         5.77279855e-02,  3.91912119e-02,  2.56211944e-02],
       [ 4.82265267e-02, -6.74389532e-02,  6.44204345e-02, ...,
         5.95007100e-02,  3.93298113e-02,  2.38431718e-02],
       [ 4.42146147e-02, -6.03095820e-02,  5.96689720e-02, ...,
         6.08009003e-02,  3.93789762e-02,  2.20194015e-02],
       ...,
       [-9.03704476e-03,  1.13378370e-01,  8.72505381e-02, ...,
        -7.04652473e-02, -2.69143945e-02, -1.99310649e-01],
       [-2.72879079e-03,  1.19000039e-01,  1.06496165e-01, ...,
        -9.01796312e-02, -2.89765200e-02, -2.14355879e-01],
       [-1.67703832e-05,  1.42264068e-01,  1.38627073e-01, ...,
        -1.05488737e-01, -2.73580771e-02, -2.35037893e-01]])

In [5]:
np.random.seed(1)
train_indices = np.random.choice(len(dataset),100,replace=False)
test_indices = [i for i in range(len(dataset)) if i not in train_indices]
train_dataset=dataset[train_indices]
test_dataset=dataset[test_indices]
print(train_dataset.shape,test_dataset.shape)

(100, 7) (9820, 7)


In [6]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(train_dataset[:,0],train_dataset[:,1],train_dataset[:,2]);
ax.scatter(test_dataset[:,0],test_dataset[:,1],test_dataset[:,2]);

<IPython.core.display.Javascript object>

In [7]:
#fig = plt.figure()
#ax = plt.axes(projection='3d')
#ax.scatter(train_dataset[:,3],train_dataset[:,4],train_dataset[:,5]);
#ax.scatter(test_dataset[:,3],test_dataset[:,4],test_dataset[:,5]);

## 2D VAE Fit to Dataset

In [8]:
def LoadDataBatch(all_points, batchsize, passthroughdim, predictive, device):
    """Sample Pair of Points from Trajectory"""
    # all_points should be of dimension: num_trajectories, numtimesteps, 1+spatialdims
    traj_ids = np.random.choice(all_points.shape[0], batchsize)
    t1_ids = np.random.choice(all_points.shape[1], batchsize)
    if predictive:
        t2_ids = np.random.choice(all_points.shape[1], batchsize)
    else:
        t2_ids = t1_ids
    return (torch.tensor(all_points[traj_ids, t1_ids,passthroughdim:], dtype=torch.float).to(device),
            torch.tensor(all_points[traj_ids, t1_ids,:passthroughdim], dtype=torch.float).to(device),
            torch.tensor(all_points[traj_ids, t2_ids,passthroughdim:], dtype=torch.float).to(device),
            torch.tensor(all_points[traj_ids, t2_ids,:passthroughdim], dtype=torch.float).to(device))

In [9]:
class Loader(object):
    def __init__(self, all_points, batchsize, passthroughdim, epochnumbatches, predictive, device):
        self.all_points = all_points
        self.batchsize = batchsize
        self.passthroughdim = passthroughdim
        self.epochnumbatches = epochnumbatches
        self.predictive = predictive
        self.device = device
        
    def __iter__(self):
        self.n = 0
        return self
    
    def __next__(self):
        if self.n >= self.epochnumbatches:
            # https://docs.python.org/3/library/exceptions.html#StopIteration
            raise StopIteration
        self.n += 1
        return LoadDataBatch(self.all_points, self.batchsize, self.passthroughdim, self.predictive, self.device)
    

device = "cuda"
  

In [10]:
dataset_dim = dataset.shape[-1]
def make_vae(latent_dim):
    input_dim = dataset_dim 
    passthrough_dim = 0
    emb_layer_widths = [1000]
    recon_layer_widths = [1000]
    dtype = torch.float
    model = ptvae.FCPassthroughVAE(input_dim, passthrough_dim, latent_dim,
        emb_layer_widths, recon_layer_widths, dtype)
    return model

In [11]:
def error(vae, dataset):
    embeds = vae.encode(torch.tensor(dataset,dtype=torch.float).to(device),
                       torch.tensor(np.zeros((len(dataset),0)),dtype=torch.float).to(device)
                       )[0].detach().cpu().numpy()
    recons = vae.decode(torch.tensor(embeds,dtype=torch.float).to(device),
              torch.tensor(np.zeros((len(embeds),0)),dtype=torch.float).to(device))[0].detach().cpu()
    rmse = np.sqrt(np.mean(np.sum(np.square(dataset - recons.numpy()),axis=1)))
    return rmse

In [12]:
# train a CurvVAE model with given beta and gamma and return
# the string name of the saved model
def train_vae_model(beta,gamma,lr,epochs,latentdim,testname,train_dataset,test_dataset,device="cpu"):
    vae = make_vae(latent_dim=latentdim).to(device)
    num_epochs = epochs
    epsilon_scale_start = 0.5
    epsilon_scale = epsilon_scale_start
    savedir  = f'{testname}lat{latentdim}_lr{lr}_gamma{gamma}_es{epsilon_scale_start}_beta{beta}_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
    print(savedir)
    train_loader = Loader(train_dataset[:,np.newaxis,:], 256, 0,10,predictive=False, device=device)
    trainer = ppttrainer.PPTVAETrainer(vae, train_loader, beta=beta, device=device,
        log_dir=savedir, lr=lr, annealingBatches=0, record_loss_every=200, loss_func = "gaussian")
    epoch = 0
    traintesterrors = []
    for _ in range(num_epochs*3):
        epoch+=1
        # up num sample points to reduce variance of curvature measure
        embst = trainer.train(second_deriv_regularizer=secreg, curvature_regularizer=gamma, epsilon_scale = epsilon_scale, num_new_samp_points=512)
        is_error = embst[0]
        if is_error:
            raise Exception
        if epoch % 10 == 0:
            traintesterrors.append((epoch,error(vae,train_dataset),error(vae,test_dataset)))
    sm.save_fcpassthrough_vae(vae, savedir)
    print(savedir)
    return savedir,traintesterrors

In [13]:
test_dataset.shape

(9820, 7)

In [14]:
testname = "overfitting/allposes"
all_models=[]
all_traintesterrors = []
for run in range(10):
    for lr in [0.00005,0.0001,0.00001]:
        for beta,gamma in [[0.00001, 1.  ],
                           [0.00001, 10.  ]
                           #[0.00001, 0.001  ],
                           #[0.00001, 0.00001],
                           #[0.001, 0],
                           #[0.00001, 0],
                           #[0.00001, 0.00005],
                           #[0.00001, 0.0001 ],
                           #[0.00001, 0.0002 ],
                           #[0.00001, 0.0003 ],
                           #[0.00001, 0.0005 ],
                           #[0.00001, 0.0007 ],
                           #[0.00001, 0.0008 ],
                           #[0.00001, 0.0009 ],
                           #[0.00001, 0.002  ],
                           #[0.00001, 0.004  ],
                           #[0.00001, 0.01  ],
                           #[0.00001, 0.1  ],
                           #[0.00001, 0],
                           #[0.0001, 0],
                           #[0.002, 0],
                           #[0.003, 0],
                           #[0.005, 0],
                           #[0.007, 0],
                           #[0.01  , 0],
                           #[0.02  , 0],
                           #[0.04  , 0],
                           #[0.1  , 0],
                           #[1.0    , 0]
                          ]:
            secreg = 0
            latentdim = 3
            epochs=800
            savedir,traintesterrors = train_vae_model(beta,gamma,lr,epochs,latentdim,testname,train_dataset,test_dataset,device=device)
            print(np.array(traintesterrors)[-10:,])
            all_models.append((beta,gamma,latentdim,lr,savedir))
            all_traintesterrors.append(traintesterrors)

overfitting/allposeslat3_lr5e-05_gamma1.0_es0.5_beta1e-05_20220718-141736
overfitting/allposeslat3_lr5e-05_gamma1.0_es0.5_beta1e-05_20220718-141736
[[2.31000000e+03 7.64736883e-02 7.47929594e-02]
 [2.32000000e+03 7.68464082e-02 7.53089523e-02]
 [2.33000000e+03 7.66678684e-02 7.47808038e-02]
 [2.34000000e+03 7.67946491e-02 7.57088079e-02]
 [2.35000000e+03 7.68898675e-02 7.46641996e-02]
 [2.36000000e+03 7.66170173e-02 7.48055768e-02]
 [2.37000000e+03 7.64558945e-02 7.46692795e-02]
 [2.38000000e+03 7.64355791e-02 7.42301400e-02]
 [2.39000000e+03 7.77930675e-02 7.54363277e-02]
 [2.40000000e+03 7.73329036e-02 7.56677375e-02]]
overfitting/allposeslat3_lr5e-05_gamma10.0_es0.5_beta1e-05_20220718-142002
overfitting/allposeslat3_lr5e-05_gamma10.0_es0.5_beta1e-05_20220718-142002
[[2.31000000e+03 1.46484434e-01 1.48850088e-01]
 [2.32000000e+03 1.46258266e-01 1.48528999e-01]
 [2.33000000e+03 1.46034429e-01 1.48239606e-01]
 [2.34000000e+03 1.45839637e-01 1.47981837e-01]
 [2.35000000e+03 1.45647848e-

overfitting/allposeslat3_lr5e-05_gamma10.0_es0.5_beta1e-05_20220718-144936
[[2.31000000e+03 1.14212579e-01 1.16911019e-01]
 [2.32000000e+03 1.13203677e-01 1.15973832e-01]
 [2.33000000e+03 1.12336282e-01 1.15293150e-01]
 [2.34000000e+03 1.11469561e-01 1.14544577e-01]
 [2.35000000e+03 1.10740012e-01 1.13707225e-01]
 [2.36000000e+03 1.09969796e-01 1.13095265e-01]
 [2.37000000e+03 1.09262932e-01 1.12386914e-01]
 [2.38000000e+03 1.08510282e-01 1.11715683e-01]
 [2.39000000e+03 1.07502864e-01 1.10848818e-01]
 [2.40000000e+03 1.06986606e-01 1.10429722e-01]]
overfitting/allposeslat3_lr0.0001_gamma1.0_es0.5_beta1e-05_20220718-145201
overfitting/allposeslat3_lr0.0001_gamma1.0_es0.5_beta1e-05_20220718-145201
[[2.31000000e+03 7.91199792e-02 7.69645956e-02]
 [2.32000000e+03 7.89103313e-02 7.63181646e-02]
 [2.33000000e+03 7.91264373e-02 7.68390907e-02]
 [2.34000000e+03 7.83976629e-02 7.57192194e-02]
 [2.35000000e+03 7.82697548e-02 7.54622364e-02]
 [2.36000000e+03 7.86576413e-02 7.56793512e-02]
 [2.37

overfitting/allposeslat3_lr0.0001_gamma1.0_es0.5_beta1e-05_20220718-152118
[[2.31000000e+03 1.21452860e-01 1.17275839e-01]
 [2.32000000e+03 1.18326521e-01 1.14121490e-01]
 [2.33000000e+03 1.16660723e-01 1.12571402e-01]
 [2.34000000e+03 1.17556440e-01 1.13773647e-01]
 [2.35000000e+03 1.78149770e-01 1.74651797e-01]
 [2.36000000e+03 1.48003665e-01 1.44410983e-01]
 [2.37000000e+03 1.33898130e-01 1.30323769e-01]
 [2.38000000e+03 1.25782689e-01 1.22403162e-01]
 [2.39000000e+03 1.21184759e-01 1.17983609e-01]
 [2.40000000e+03 1.17874699e-01 1.14889383e-01]]
overfitting/allposeslat3_lr0.0001_gamma10.0_es0.5_beta1e-05_20220718-152344
overfitting/allposeslat3_lr0.0001_gamma10.0_es0.5_beta1e-05_20220718-152344
[[2.31000000e+03 2.01258432e-01 2.05530456e-01]
 [2.32000000e+03 1.96931878e-01 2.01487941e-01]
 [2.33000000e+03 1.93803116e-01 1.98215198e-01]
 [2.34000000e+03 1.91329622e-01 1.95921584e-01]
 [2.35000000e+03 1.89387655e-01 1.94024764e-01]
 [2.36000000e+03 1.87804636e-01 1.92326798e-01]
 [2.

overfitting/allposeslat3_lr0.0001_gamma10.0_es0.5_beta1e-05_20220718-155309
[[2.31000000e+03 1.18546424e-01 1.14033582e-01]
 [2.32000000e+03 1.18205560e-01 1.13581935e-01]
 [2.33000000e+03 1.17783904e-01 1.13154907e-01]
 [2.34000000e+03 1.17405824e-01 1.12704424e-01]
 [2.35000000e+03 1.16974332e-01 1.12257157e-01]
 [2.36000000e+03 1.16549910e-01 1.11795333e-01]
 [2.37000000e+03 1.16080267e-01 1.11168879e-01]
 [2.38000000e+03 1.15547942e-01 1.10620266e-01]
 [2.39000000e+03 1.15018172e-01 1.10089956e-01]
 [2.40000000e+03 1.14463507e-01 1.09663326e-01]]
overfitting/allposeslat3_lr1e-05_gamma1.0_es0.5_beta1e-05_20220718-155536
overfitting/allposeslat3_lr1e-05_gamma1.0_es0.5_beta1e-05_20220718-155536
[[2.31000000e+03 1.78359530e-01 1.82579436e-01]
 [2.32000000e+03 1.78973075e-01 1.83280170e-01]
 [2.33000000e+03 1.78699709e-01 1.82698154e-01]
 [2.34000000e+03 1.82259477e-01 1.86496610e-01]
 [2.35000000e+03 1.77431773e-01 1.81771976e-01]
 [2.36000000e+03 1.85098990e-01 1.89668514e-01]
 [2.370

overfitting/allposeslat3_lr1e-05_gamma1.0_es0.5_beta1e-05_20220718-162521
[[2.31000000e+03 1.49871549e-01 1.49719375e-01]
 [2.32000000e+03 1.53679708e-01 1.54898604e-01]
 [2.33000000e+03 1.50974848e-01 1.51761392e-01]
 [2.34000000e+03 1.49692604e-01 1.49796949e-01]
 [2.35000000e+03 1.51016823e-01 1.52209375e-01]
 [2.36000000e+03 1.47345173e-01 1.47562004e-01]
 [2.37000000e+03 1.43730904e-01 1.44013247e-01]
 [2.38000000e+03 1.42065187e-01 1.41960341e-01]
 [2.39000000e+03 1.42267864e-01 1.42242876e-01]
 [2.40000000e+03 1.39137352e-01 1.38863066e-01]]
overfitting/allposeslat3_lr1e-05_gamma10.0_es0.5_beta1e-05_20220718-162748
overfitting/allposeslat3_lr1e-05_gamma10.0_es0.5_beta1e-05_20220718-162748
[[2.31000000e+03 2.09459365e-01 2.10960552e-01]
 [2.32000000e+03 2.08935163e-01 2.10546563e-01]
 [2.33000000e+03 2.09764942e-01 2.11465470e-01]
 [2.34000000e+03 2.07327966e-01 2.09271057e-01]
 [2.35000000e+03 2.05203413e-01 2.06784292e-01]
 [2.36000000e+03 2.09005258e-01 2.11009506e-01]
 [2.370

In [15]:
np.savez(f'justquat_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}', all_models=all_models,all_traintesterrors=all_traintesterrors)

In [16]:
dat = np.load("justquat_20220718-072431.npz")
all_models = dat["all_models"]
all_traintesterrors = dat["all_traintesterrors"]

In [17]:
#savedirs = [
#"overfitting/testcodelat2_lr0.0001_gamma0_es1.0_beta1e-05_20220714-180311",
#"overfitting/testcodelat2_lr0.0001_gamma0.001_es1.0_beta1e-05_20220714-180335",
#"overfitting/testcodelat2_lr0.0001_gamma1_es1.0_beta1e-05_20220714-180359",
#"overfitting/testcodelat2_lr0.0001_gamma0_es1.0_beta0.001_20220714-180422",
#"overfitting/testcodelat2_lr0.0001_gamma0.001_es1.0_beta0.001_20220714-180447",
#"overfitting/testcodelat2_lr0.0001_gamma1_es1.0_beta0.001_20220714-180510",
#"overfitting/testcodelat2_lr0.0001_gamma0_es1.0_beta1.0_20220714-180535",
#"overfitting/testcodelat2_lr0.0001_gamma0.001_es1.0_beta1.0_20220714-180558",
#"overfitting/testcodelat2_lr0.0001_gamma1_es1.0_beta1.0_20220714-180621"
#]

In [18]:
all_traintesterrors

array([[[1.00000000e+01, 2.23937707e-01, 2.29230250e-01],
        [2.00000000e+01, 2.24212578e-01, 2.29088334e-01],
        [3.00000000e+01, 2.23561953e-01, 2.27910855e-01],
        ...,
        [2.38000000e+03, 7.17805704e-02, 7.43848674e-02],
        [2.39000000e+03, 7.06790391e-02, 7.24740187e-02],
        [2.40000000e+03, 7.14951753e-02, 7.61652288e-02]],

       [[1.00000000e+01, 2.36374661e-01, 2.42561492e-01],
        [2.00000000e+01, 2.29465110e-01, 2.34943277e-01],
        [3.00000000e+01, 2.25719855e-01, 2.30349087e-01],
        ...,
        [2.38000000e+03, 1.44095035e-02, 7.54619802e-02],
        [2.39000000e+03, 1.45703234e-02, 7.54233409e-02],
        [2.40000000e+03, 1.43128204e-02, 7.53177849e-02]],

       [[1.00000000e+01, 2.54546131e-01, 2.58181900e-01],
        [2.00000000e+01, 2.31519157e-01, 2.36346032e-01],
        [3.00000000e+01, 2.27138995e-01, 2.31846133e-01],
        ...,
        [2.38000000e+03, 1.57339053e-02, 8.90316542e-02],
        [2.39000000e+03, 1.57

In [19]:
[am[3] for am in all_models]

['5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '0.0001',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '1e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5e-05',
 '5

In [20]:
len(all_models)

810

In [21]:
plt.figure()
numpointstotal = 0
for i,dat in enumerate(all_traintesterrors):
    (beta,gamma,latentdim,lr,savedir)=all_models[i]
    label="b%s_g%s" % (beta,gamma)
    dat = np.array(dat)
    okinds = (dat[:,1] < 0.3) * np.arange(len(dat)) > (len(dat)-2)
    train = dat[okinds,1]
    test = dat[okinds,2]
    #if lr == "0.0001" or lr == "5e-05":
    #    continue
    if gamma == "0.1":
        plt.scatter(train,test,label=label,s=3,c="blue")
    elif gamma == "0":
        plt.scatter(train,test,label=label,s=3,c="red")
    else:
        plt.scatter(train,test,label=label,s=3,c="k")
    numpointstotal+=1
print(numpointstotal)
#plt.xlim(0,0.25)
#plt.ylim(0,0.25)
#plt.plot([0,0.25],[0,0.25],linestyle="--")
#for pcatrain, pcatest in [(0.10528128829504901, 0.10464429348233918),
#                          (0.14522424739834378, 0.14321458052304378),
#                         (0.22419371587397705, 0.2280505547284446)]:
#    plt.scatter(pcatrain,pcatest,s=50,c="blue")
#    #plt.plot((0.01,0.2),(0.10464429348233918,0.10464429348233918))
#plt.legend()
plt.axis("equal")
plt.xlabel("Training RMSE")
plt.ylabel("Test RMSE")
plt.savefig("FittingAllPosesTrainSize100.png")

<IPython.core.display.Javascript object>

810


In [22]:

plt.figure(figsize=(10,10))
for i,dat in enumerate(all_traintesterrors):
    (beta,gamma,latentdim,lr,savedir)=all_models[i]
    label="b%s_g%s" % (beta,gamma)
    dat = np.array(dat)
    okinds = (dat[:,1] < 0.3) * np.arange(len(dat)) > len(dat)*93/100
    train = dat[okinds,1]
    test = dat[okinds,2]
    if gamma != 0:
        plt.plot(np.ones_like(test) * gamma,test,label=label)
        plt.plot(np.ones_like(test) * gamma,train,label=label)
    else:
        pass#plt.plot(train,test,label=label)
plt.xscale("log")
plt.ylim(0,0.25)
for pcatrain, pcatest in [(0.08209298876189912, 0.08050582337947013),
                          (0.08209298876189912, 0.08050582337947013)]
    plt.plot([0.0001,0.1],[pcatrain,pcatrain],linestyle="--")
    plt.plot([0.0001,0.1],[pcatest,pcatest],linestyle="--")

plt.legend()

SyntaxError: invalid syntax (<ipython-input-22-ca6b2c2459cd>, line 17)